<a href="https://colab.research.google.com/github/ELehmann91/NLP-Contract-Analysis/blob/master/coop_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1lG3ReZc9ESyVPsstjuu5ek73u6vVsi3X)


# Set-Up

Setting up the environment in Colab to run various experiments, note the cuda version of spacy-pytorch-transformers is being downloaded

https://github.com/explosion/spacy-transformers/issues/39

In [0]:
%%capture
#!python -m spacy download en_trf_bertbaseuncased_lg
#!python -m spacy download de_trf_bertbasecased_lg
!pip install spacy==2.2.0
!pip install torch==1.1.0
#!pip install spacy-transformers
#!python -m spacy download de_trf_bertbasecased_lg
!pip install PyPDF2
!sudo apt install poppler-utils
!pip install pdf2image
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install spacy
!pip install spacy-langdetect
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

In [0]:
#!pip install gputil
#!pip install torch==1.1.0
#!pip install spacy-pytorch-transformers[cuda100]==0.2.0
#!pip install spacy==2.2.0
#!python -m spacy download de_pytt_bertbasecased_lg #spacy 2.1.9

You will need to **restart runtime after these installs** to reinstatiate the environment/directory

In [0]:
import spacy
import torch
import numpy
from numpy.testing import assert_almost_equal
from scipy.spatial import distance
from spacy.util import minibatch
import random
#import cupy
import numpy as np
import pandas as pd
import os
import json
import re
import pytesseract
from PIL import Image
import joblib
import pickle
from tqdm import tqdm

In [0]:
import warnings
warnings.filterwarnings("ignore")

## Mount Drive

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
path = '/content/gdrive/Shared drives/FS-AI Base Data/innolytiq_pdf/'
file_list = os.listdir(path)
len(file_list)

150

## OCR

writes text file for pdfs

In [0]:
import os
import tempfile
from pdf2image import convert_from_path

def OCR_pdf(path_,file_,no_pages=10):
    filename = path_ + file_ +'/' + file_ +'.pdf' #'target.pdf'
    if os.path.exists(path_+ file_+'/extracted_text.txt') == False:

        # import pdf to img
        with tempfile.TemporaryDirectory() as path:
            # mag page number
            images_from_path = convert_from_path(filename, output_folder=path, last_page=no_pages, first_page =0)
        #filename
        base_filename  =  os.path.splitext(os.path.basename(filename))[0] + '.jpg'     
        #create folder for images
        img_dir = path_ + file_ + '/pdf2img'
        try:
          os.mkdir(img_dir)
        except FileExistsError:
          next
          #print('check')

        text = ''
        for i,page in enumerate(images_from_path):
            #save as jpg
            page.save(os.path.join(img_dir, base_filename+str(i)), 'JPEG')
            #read jpg and extract text
            text += pytesseract.image_to_string(Image.open(os.path.join(img_dir, base_filename+str(i)))) + ' '
            #
        file1 = open(path_+ file_+'/extracted_text.txt','w') 
        file1.writelines(text) 
        file1.close()

        #print(filename,'pages:',len(images_from_path))
    else:
      next
      #print(filename,' allready exists')


In [0]:
path_ = '/content/gdrive/Shared drives/FS-AI Base Data/innolytiq_pdf/'
file_list = os.listdir(path_)

for file_ in tqdm(file_list):
    OCR_pdf(path_,file_)


100%|██████████| 149/149 [00:00<00:00, 6711.17it/s]

## Build Dataframe from data.json

In [0]:
# name list contains lists of columns
name_list = []

with open(path + file_list[0]+'/data.json', encoding="utf8") as json_file:
    data = json.load(json_file)
    for line in data['output']:
        try: #line['members'] is not None:
            name_list.append(line['name'])
        except:
            print(line)

In [0]:
# create empty dataframe
df_ = pd.DataFrame(index=file_list, columns=name_list)
df_ = df_.fillna(0) # with 0s rather than NaNs

In [0]:
# read data from json to dataframe
for file in tqdm(file_list):
    with open(path+file+'/data.json', encoding="utf8") as json_file:
        data = json.load(json_file)
        #print(data)
        try:
            for line in data['output']:
                try: # line['members'] is not None:
                    df_.loc[df_.index==file,line['name']] = line['data'][0]['value']
                except:
                    pass
                    #print(line['name'])
        except KeyError:
            print(file)


100%|██████████| 149/149 [00:02<00:00, 58.74it/s]

## Read PDF

In [0]:
df_['pdf_text'] = 'leer'
i=1
for file_ in tqdm(file_list):
    #print(i)
    try:
      with open(path+file_+'/extracted_text.txt',"r") as file1:
        text = file1.readlines()
        text = ''.join([t.replace('\n',' ') for t in text])
        file1.close() 
        
        #print('name:',file_,'words:',len(text.split()))
        df_['pdf_text'][df_.index==file_] = text
        i+=1
    except:
        df_['pdf_text'][df_.index==file_] = 'leer'
        print('cant read '+file_)


100%|██████████| 149/149 [00:48<00:00,  2.69it/s]


In [0]:
print(df_.shape)
df_[:2]

(149, 24)


,006_issue_date,009_nominal_amount,008_maturity_date,010_redemption_formula,011_seniority,012_business_day_convention,014_business_day_calendar,017_first_payment_date,018_payment_dates,021_formula,002_issuer_name,020_paragraph_489,004_identifiers,007_issue_price,003_product_subclass,015_payment_frequency,005_currency,013_day_count_convention,001_type,004_wkn,004_isin,023_amendment,022_legislation,pdf_text
erfurter_bank_eg.pdf,22.02.2013,"5.000.000,00",22.02.2022,Nennwert,,,,22.02.2014,22.02.,"2,34 Prozent",Die Erfurter Bank eG,,0,"100,00%",Schuldschein,1,EUR,act/act,Fixed,,,no_amendment,Ger,NU IV UAAYY G SCHULDSCHEIN Die Erfurter Bank...
de000dr0q3k7_en.pdf,23 July 2008,"100,000,000",21 September 2012,100%,,,,21 September 2009,21 September 2009 \n21 September 2010 \n21 Sep...,10 years EURO - Swap minus 2 years EURO Swap a...,"Dresdner Bank Aktiengesellschaft, Frankfurt am...",,0,100,other Loan,1,EUR,,Floating,DR0Q3K,DE 000 DR0 Q3K 7,no_amendment,Ger,Dresdner Bank AG Private Investor Products +4...


## Preprocess text

In [0]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

sw_en = stopwords.words('english')
sw_de = stopwords.words('german')
sw_en.extend(sw_de)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
def propro(sen):
  sen = sen.lower()
  sen = sen.replace('\n',' ')
  sen = sen.replace('%','prozent')
  sen = re.sub('[^A-Za-zÄÖÜßäöüß]+', ' ',sen)
  sen = ' '.join([w for w in sen.split() if w not in sw_en])
  return sen

df_['pdf_text_prepro'] = df_['pdf_text'].apply(lambda x: propro(x))

# Model

## Classification - LogReg

### 003_product_subclass'

In [0]:
df_['003_product_subclass'] = df_['003_product_subclass'].apply(lambda x: 'no_class' if x == '' else x)
df_['003_product_subclass'].value_counts()

Bond                                             68
Schuldschein                                     26
other Loan                                       19
Covered Bond                                     18
no_class                                          6
SCHULDVERSCHREIBUNGSTYP                           3
Term Loan                                         2
(the "Securities") \nSchuldverschreibungen"       2
TYPE OF SECURITIES  \nSCHULDVERSCHREIBUNGSTYP     2
chuldverschreibungen"                             1
"Schuldverschreibungen"                           1
Anleihe                                           1
Name: 003_product_subclass, dtype: int64

In [0]:
bool_col = (np.where(df_['003_product_subclass']=='Bond',True,False) +
            np.where(df_['003_product_subclass']=='Schuldschein',True,False) +
            np.where(df_['003_product_subclass']=='other Loan',True,False) +
            np.where(df_['003_product_subclass']=='Covered Bond',True,False) +
            np.where(df_['003_product_subclass']=='no_class',True,False))

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df_['pdf_text_prepro'][bool_col], 
                                                    df_['003_product_subclass'][bool_col],
                                                    random_state=42,
                                                    #stratify=y,
                                                    test_size=0.2)

In [0]:
logreg = Pipeline([('vect', CountVectorizer(max_features=5000)),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression( C=1e7)),
               ])
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_train)
y_pred_ = logreg.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_train))
print('accuracy %s' % accuracy_score(y_pred_, y_test))

accuracy 1.0
accuracy 0.8571428571428571


In [0]:
for i in range(0,5):
  print('prediction:',y_pred_[i],'/// true_val:',y_test[i])

prediction: Bond /// true_val: Bond
prediction: Bond /// true_val: Bond
prediction: Schuldschein /// true_val: Schuldschein
prediction: Bond /// true_val: Bond
prediction: Bond /// true_val: Bond


In [0]:
# save the model to disk
save_path = '/content/gdrive/Shared drives/FS-AI Base Data/models/'
filename = '003_product_subclass_model.joblib'
#joblib.dump(logreg, save_path+filename)
pickle.dump(logreg, open(save_path+filename, 'wb'))
 

# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)

In [0]:
save_path+filename

'/content/gdrive/Shared drives/FS-AI Base Data/models/003_product_subclass_model.joblib'

In [0]:
#loaded_model = pickle.load(open(save_path+filename, 'rb'))
loaded_model = joblib.load(save_path+filename)
pred = loaded_model.predict(df_['pdf_text_prepro'])
result = loaded_model.score(df_['pdf_text_prepro'], df_['003_product_subclass'])
result

0.8926174496644296

In [0]:
pred = loaded_model.predict(df_['pdf_text_prepro'][1])
pred

ValueError: ignored

### 011_seniority

In [0]:
df_['011_seniority'] = df_['011_seniority'].apply(lambda x: 'no_class' if x == '' else x)
df_['011_seniority'].value_counts()

no_class            79
Senior Secured      29
Senior              29
Senior Unsecured     9
Unsecured            2
Subordinated         1
Name: 011_seniority, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_['pdf_text_prepro'], 
                                                    df_['011_seniority'],
                                                    random_state=42,
                                                    #stratify=y,
                                                    test_size=0.2)

logreg = Pipeline([('vect', CountVectorizer(max_features=5000)),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=1e7)),
               ])
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_train)
y_pred_ = logreg.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_train))
print('accuracy %s' % accuracy_score(y_pred_, y_test))

for i in range(0,5):
  print('prediction:',y_pred_[i],'/// true_val:',y_test[i])

# save the model to disk
save_path = '/content/gdrive/Shared drives/FS-AI Base Data/models/'
filename = '011_seniority_model.sav'
pickle.dump(logreg, open(save_path+filename, 'wb'))

accuracy 1.0
accuracy 0.6333333333333333
prediction:  /// true_val: 
prediction:  /// true_val: 
prediction: Senior Secured /// true_val: Senior
prediction:  /// true_val: 
prediction: Senior /// true_val: Senior


### 012_business_day_convention

In [0]:
df_['012_business_day_convention'] = df_['012_business_day_convention'].apply(lambda x: 'no_class' if x == '' else x)
df_['012_business_day_convention'].value_counts()

no_class                         70
Following unadjusted             41
Unadjusted                       17
Modified following adjusted       7
Following adjusted                6
Modified following unadjusted     4
Adjusted                          3
Modified Following                1
Name: 012_business_day_convention, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_['pdf_text_prepro'], 
                                                    df_['012_business_day_convention'],
                                                    random_state=42,
                                                    #stratify=y,
                                                    test_size=0.2)

logreg = Pipeline([('vect', CountVectorizer(max_features=5000)),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=1e7)),
               ])
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_train)
y_pred_ = logreg.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_train))
print('accuracy %s' % accuracy_score(y_pred_, y_test))

for i in range(0,5):
  print('prediction:',y_pred_[i],'/// true_val:',y_test[i])

# save the model to disk
save_path = '/content/gdrive/Shared drives/FS-AI Base Data/models/'
filename = '012_business_day_convention_model.sav'
pickle.dump(logreg, open(save_path+filename, 'wb'))
 

accuracy 1.0
accuracy 0.6666666666666666
prediction: no_class /// true_val: no_class
prediction: no_class /// true_val: no_class
prediction: Following adjusted /// true_val: Following adjusted
prediction: no_class /// true_val: no_class
prediction: Following unadjusted /// true_val: Unadjusted


### 022_legislation

In [0]:
df_['022_legislation'] = df_['022_legislation'].apply(lambda x: 'no_class' if x == '' else x)
df_['022_legislation'].value_counts()

no_class                    82
Ger                         41
German Law                   8
Netherlands                  7
NLD                          5
London,                      3
Stockholm, London            1
ederlandse \nNetherlands     1
German law.                  1
Name: 022_legislation, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_['pdf_text_prepro'], 
                                                    df_['022_legislation'],
                                                    random_state=42,
                                                    #stratify=y,
                                                    test_size=0.2)

logreg = Pipeline([('vect', CountVectorizer(max_features=5000)),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=1e7)),
               ])
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_train)
y_pred_ = logreg.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_train))
print('accuracy %s' % accuracy_score(y_pred_, y_test))

for i in range(0,5):
  print('prediction:',y_pred_[i],'/// true_val:',y_test[i])

# save the model to disk
save_path = '/content/gdrive/Shared drives/FS-AI Base Data/models/'
filename = '022_legislation_model.sav'
pickle.dump(logreg, open(save_path+filename, 'wb'))
 

accuracy 1.0
accuracy 0.5
prediction: no_class /// true_val: no_class
prediction: Ger /// true_val: Ger
prediction: no_class /// true_val: no_class
prediction: no_class /// true_val: German Law
prediction: no_class /// true_val: Netherlands


### 020_paragraph_489

In [0]:
df_['020_paragraph_489'] = df_['020_paragraph_489'].apply(lambda x: 'no_class' if x == '' else x)
df_['020_paragraph_489'].value_counts()

no_class                      115
Excluded                       25
Not Excluded                    8
10 Jahren gemäß § 489 Abs.      1
Name: 020_paragraph_489, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_['pdf_text_prepro'], 
                                                    df_['020_paragraph_489'],
                                                    random_state=42,
                                                    #stratify=y,
                                                    test_size=0.2)

logreg = Pipeline([('vect', CountVectorizer(max_features=5000)),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=1e7)),
               ])
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_train)
y_pred_ = logreg.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_train))
print('accuracy %s' % accuracy_score(y_pred_, y_test))

for i in range(0,5):
  print('prediction:',y_pred_[i],'/// true_val:',y_test[i])

# save the model to disk
save_path = '/content/gdrive/Shared drives/FS-AI Base Data/models/'
filename = '020_paragraph_489_model.sav'
pickle.dump(logreg, open(save_path+filename, 'wb'))
 

accuracy 1.0
accuracy 0.8
prediction: no_class /// true_val: no_class
prediction: no_class /// true_val: no_class
prediction: no_class /// true_val: no_class
prediction: no_class /// true_val: no_class
prediction: no_class /// true_val: no_class


### 023_amendment

In [0]:
df_['023_amendment'] = df_['023_amendment'].apply(lambda x: 'no_class' if x == '' else x)
df_['023_amendment'].value_counts()

no_amendment    79
no_class        69
amendment        1
Name: 023_amendment, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_['pdf_text_prepro'], 
                                                    df_['023_amendment'],
                                                    random_state=42,
                                                    #stratify=y,
                                                    test_size=0.2)

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

from sklearn.metrics import accuracy_score, confusion_matrix

logreg = Pipeline([('vect', CountVectorizer(max_features=5000)),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=1e7)),
               ])
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_train)
y_pred_ = logreg.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_train))
print('accuracy %s' % accuracy_score(y_pred_, y_test))

for i in range(0,5):
  print('prediction:',y_pred_[i],'/// true_val:',y_test[i])

# save the model to disk
save_path = '/content/gdrive/Shared drives/FS-AI Base Data/models/'
filename = '023_amendment_model.sav'
pickle.dump(logreg, open(save_path+filename, 'wb'))
 

accuracy 1.0
accuracy 0.7666666666666667
prediction: no_amendment /// true_val: no_amendment
prediction: no_amendment /// true_val: no_amendment
prediction: no_class /// true_val: no_class
prediction: no_class /// true_val: no_class
prediction: no_class /// true_val: no_amendment


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### 005_currency

In [0]:
df_['005_currency'] = df_['005_currency'].apply(lambda x: 'no_class' if x == '' else x)
df_['005_currency'].value_counts()

EUR             80
Euro            26
USD             11
Euro (€)         7
no_class         7
("EUR")          4
Euro ("EUR")     2
UR"              1
('NZD")          1
Euro ("EUR')     1
CHF              1
GBP              1
Euro (EtlR)      1
SEK              1
("EUR" o         1
AUD              1
GBP ("GBP")      1
other            1
(ELIR)           1
Name: 005_currency, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_['pdf_text_prepro'], 
                                                    df_['005_currency'],
                                                    random_state=42,
                                                    #stratify=y,
                                                    test_size=0.2)

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

from sklearn.metrics import accuracy_score, confusion_matrix

logreg = Pipeline([('vect', CountVectorizer(max_features=5000)),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=1e7)),
               ])
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_train)
y_pred_ = logreg.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_train))
print('accuracy %s' % accuracy_score(y_pred_, y_test))

for i in range(0,5):
  print('prediction:',y_pred_[i],'/// true_val:',y_test[i])

# save the model to disk
save_path = '/content/gdrive/Shared drives/FS-AI Base Data/models/'
filename = '005_currency_model.sav'
pickle.dump(logreg, open(save_path+filename, 'wb'))
 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


accuracy 1.0
accuracy 0.5666666666666667
prediction: EUR /// true_val: EUR
prediction: EUR /// true_val: EUR
prediction: USD /// true_val: USD
prediction: Euro /// true_val: Euro ("EUR")
prediction: Euro (EtlR) /// true_val: EUR


### 015_payment_frequency

In [0]:
df_['015_payment_frequency'] = df_['015_payment_frequency'].apply(lambda x: 'no_class' if x == '' else x)
df_['015_payment_frequency'].value_counts()

1           109
4            14
0            14
no_class      7
2             5
Name: 015_payment_frequency, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_['pdf_text_prepro'], 
                                                    df_['015_payment_frequency'],
                                                    random_state=42,
                                                    #stratify=y,
                                                    test_size=0.2)

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

from sklearn.metrics import accuracy_score, confusion_matrix

logreg = Pipeline([('vect', CountVectorizer(max_features=5000)),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=1e7)),
               ])
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_train)
y_pred_ = logreg.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_train))
print('accuracy %s' % accuracy_score(y_pred_, y_test))

for i in range(0,5):
  print('prediction:',y_pred_[i],'/// true_val:',y_test[i])

# save the model to disk
save_path = '/content/gdrive/Shared drives/FS-AI Base Data/models/'
filename = '015_payment_frequency_model.sav'
pickle.dump(logreg, open(save_path+filename, 'wb'))
 

accuracy 1.0
accuracy 0.9333333333333333
prediction: 1 /// true_val: 1
prediction: no_class /// true_val: 1
prediction: 4 /// true_val: 4
prediction: 1 /// true_val: 2
prediction: 1 /// true_val: 1


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


### 014_business_day_calendar	

In [0]:
df_['014_business_day_calendar'] = df_['014_business_day_calendar'].apply(lambda x: 'no_class' if x == '' else x)
df_['014_business_day_calendar'].value_counts()

TARGET2                         81
no_class                        37
TARGET                          21
'l'ARGE'l'a                      1
Euromarket\nTARGET2              1
Clearing Sy\n(TARGET2)           1
, TARGET 2                       1
TARGET2,                         1
London and TARGET2               1
TARGET a                         1
TARGET & Sydney                  1
London and TARGET                1
New York, London and TARGET2     1
Name: 014_business_day_calendar, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_['pdf_text_prepro'], 
                                                    df_['014_business_day_calendar'],
                                                    random_state=42,
                                                    #stratify=y,
                                                    test_size=0.2)

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

from sklearn.metrics import accuracy_score, confusion_matrix

logreg = Pipeline([('vect', CountVectorizer(max_features=5000)),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=1e7)),
               ])
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_train)
y_pred_ = logreg.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_train))
print('accuracy %s' % accuracy_score(y_pred_, y_test))

for i in range(0,5):
  print('prediction:',y_pred_[i],'/// true_val:',y_test[i])

# save the model to disk
save_path = '/content/gdrive/Shared drives/FS-AI Base Data/models/'
filename = '014_business_day_calendar_model.sav'
pickle.dump(logreg, open(save_path+filename, 'wb'))
 

accuracy 0.9915966386554622
accuracy 0.7666666666666667
prediction: no_class /// true_val: no_class
prediction: no_class /// true_val: no_class
prediction: TARGET2 /// true_val: TARGET2
prediction: TARGET2 /// true_val: TARGET2
prediction: TARGET2 /// true_val: TARGET2


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


### 013_day_count_convention




In [0]:
df_['013_day_count_convention'] = df_['013_day_count_convention'].apply(lambda x: 'no_class' if x == '' else x)
df_['013_day_count_convention'].value_counts()

Act/act ICMA               44
no_class                   32
Actual/Actual (ICMA)       20
30/360                     15
Act/360                     9
act/act                     7
301360                      5
ActuaVActual (ICMA)         3
Actual / Actual (ICMA)      3
30E/360                     2
Actual/360                  2
"Accrual Period             1
ActuaV360                   1
Actual/Actual               1
Act/365 L, Act/act ICMA     1
Act/act ISDA                1
Actual/365                  1
ActuaU360                   1
Name: 013_day_count_convention, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_['pdf_text_prepro'], 
                                                    df_['013_day_count_convention'],
                                                    random_state=42,
                                                    #stratify=y,
                                                    test_size=0.2)

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

from sklearn.metrics import accuracy_score, confusion_matrix

logreg = Pipeline([('vect', CountVectorizer(max_features=5000)),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=1e7)),
               ])
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_train)
y_pred_ = logreg.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_train))
print('accuracy %s' % accuracy_score(y_pred_, y_test))

for i in range(0,5):
  print('prediction:',y_pred_[i],'/// true_val:',y_test[i])

# save the model to disk
save_path = '/content/gdrive/Shared drives/FS-AI Base Data/models/'
filename = '013_day_count_convention_model.sav'
pickle.dump(logreg, open(save_path+filename, 'wb'))
 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


accuracy 1.0
accuracy 0.4666666666666667
prediction: no_class /// true_val: no_class
prediction: no_class /// true_val: no_class
prediction: Act/360 /// true_val: Actual/360
prediction: Actual/Actual (ICMA) /// true_val: Actual/360
prediction: 301360 /// true_val: Actual/Actual (ICMA)


###	001_type

In [0]:
df_['001_type'] = df_['001_type'].apply(lambda x: 'no_class' if x == '' else x)
df_['001_type'].value_counts()

Fixed         96
Floating      44
no_class       4
Zero           3
Fixed Rate     2
Name: 001_type, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_['pdf_text_prepro'], 
                                                    df_['001_type'],
                                                    random_state=42,
                                                    #stratify=y,
                                                    test_size=0.2)

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

from sklearn.metrics import accuracy_score, confusion_matrix

logreg = Pipeline([('vect', CountVectorizer(max_features=5000)),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=1e7)),
               ])
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_train)
y_pred_ = logreg.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_train))
print('accuracy %s' % accuracy_score(y_pred_, y_test))

for i in range(0,5):
  print('prediction:',y_pred_[i],'/// true_val:',y_test[i])

# save the model to disk
save_path = '/content/gdrive/Shared drives/FS-AI Base Data/models/'
filename = '001_type_model.sav'
pickle.dump(logreg, open(save_path+filename, 'wb'))
 

accuracy 0.9915966386554622
accuracy 0.8333333333333334
prediction: Fixed /// true_val: Fixed
prediction: Floating /// true_val: Floating
prediction: Floating /// true_val: Floating
prediction: Fixed /// true_val: Floating
prediction: Fixed /// true_val: Fixed Rate


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


## Extraction - Manul Filter

In [0]:
import os
import json
import pandas as pd
import re
import spacy
nlp_en = spacy.load("en_core_web_sm")
nlp_de = spacy.load("de_core_news_sm")
from spacy.matcher import PhraseMatcher
from spacy_langdetect import LanguageDetector

In [0]:
def multi_corpus(text):
  '''
  check if german or english and use spacy language model accordingly 
  return spacy doc
  '''
  nlp = spacy.load("en_core_web_sm")
  nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)
  doc = nlp(text)
  # document level language detection. Think of it like average language of the document!
  if doc._.language['language'] == 'en':
    #print('english detected')
    doc = nlp_en(text)
  elif doc._.language['language'] == 'de':
    #print('german detected')
    doc = nlp_en(text)
  else:
    print('not found', doc._.language['language'])
  return doc


### 007_issue_price

In [0]:
df_['007_issue_price'].value_counts()

100                                                                                                                                                                                                           37
100.00                                                                                                                                                                                                        27
                                                                                                                                                                                                              26
100,00                                                                                                                                                                                                         3
100                                                                                                                                                                 

In [0]:
def find_ident(doc,ident_list,window_w=150):
  '''
  input spacy doc and list with string identifiers
  loops throgh the document and searches for identifiers
  if found return a window of words
  '''
  windows = []
  for i, token in enumerate(doc):
      if token.text in ident_list:
        window = [tok for tok in doc[i-1:i+window_w]]
        windows.extend(window)
      if doc[i-1].text +' '+ token.text in ident_list:
        window = [tok for tok in doc[i-2:i+window_w]]
        windows.extend(window)
      #print(token.text, token.pos_, token.tag_, token.dep_, token.shape_)
  return windows

In [0]:
def find_number(doc,length=8,min=60,max=200):
  '''
  scans through a documents for numers between 60 and 200
  '''
  for tok in doc:
    tag = tok.tag_ in ['CD','CARD']
    lgt = len(tok.text)<length
    try:
      num_o = min < float(re.sub('[^0-9.,]', ' ',tok.text).replace(',','.')) < max
    except:
      bum_o = False
      next
    if tag and lgt and num_o:
      return tok.text

In [0]:
test = 'Der Ausgabepreis 100,000,0.00 beträgt 101,50 Euro und fünfzig'
doc = nlp_de(test)
find_number(doc)

'101,50'

In [0]:
#identifier
issue_price_ident = ["Ausgabepreis", "Issue Price", "Emissionskurs"]

In [0]:
issue_no = []
for i in range(0,len(df_)):
  text = df_['pdf_text'][i]
  doc = multi_corpus(text)
  window = find_ident(doc,issue_price_ident)
  no = find_number(window)
  #print(i,df_.index[i],window,no,df_['007_issue_price'][i])
  issue_no.append(no)


In [0]:
a=0
for i in range(0,len(issue_no)):

  if issue_no[i]==df_['007_issue_price'][i]:
    a +=1

a/len(issue_no)

0.5167785234899329

### 004_wkn	

In [0]:
df_['004_wkn'].value_counts()

           87
DR0Q3K      3
DR0 Q16     2
DR4WDY      2
DR9RHD      2
DR0 QUQ     2
DL19TJ      1
DR5 DQ4     1
DR6YTN      1
DL19T4      1
DL19S3      1
DL19TG      1
DL19T5      1
DL19S4      1
A1ZW6A      1
HSH3J4      1
DB5DCC      1
DL19ST      1
DL19T9      1
DR5XZF      1
DR0Q3D      1
A2E4ZD      1
A1AZR0      1
DL19T0      1
CB83GN      1
A0FAG7      1
DL19T7      1
DR1 VLH     1
DL19SZ      1
DR0 RVP     1
DR0Q0E      1
DR0PUN      1
DL19UH      1
DL19SK      1
DL19UY      1
DL19UL      1
DL19T8      1
DR0FL3      1
DL19TK      1
DL19UK      1
DL19TF      1
CZ3 1P5     1
DL19S9      1
DE04XK      1
DL19TL      1
A0E83L      1
DL19TE      1
CB8 3HP     1
DB5DBZ      1
A1H3V5      1
A2BPC4      1
DR6 D0M     1
DL19UU      1
CZ2 25Q     1
DR1 WF5     1
DL19TH      1
757 271     1
Name: 004_wkn, dtype: int64

### 004_isin

In [0]:
df_['004_isin'].value_counts()

                      15
X51992160827           3
DE 000 DR0 Q16 9       2
DE 000 DR0 Q3K 7       2
DE 000 DR0 QUQ 5       2
                      ..
DE000A0FAG76           1
DE000DR0PUN4           1
:DEOCOAOES3I5          1
DE000A2E4ZD1           1
I\nXS 142037 955 I     1
Name: 004_isin, Length: 126, dtype: int64

### 006_issue_date	

In [0]:
df_['006_issue_date'].value_counts()

                    18
13 November 2018     2
24 March 2017        2
27 July 2018         2
23 July 2008         2
                    ..
August 6, 2007       1
29th may 2007        1
18 April 2019        1
30. Juni 2011        1
March 22, 2010       1
Name: 006_issue_date, Length: 120, dtype: int64

### 009_nominal_amount

In [0]:
df_['009_nominal_amount'].value_counts()

10,000,000       14
20,000,000        6
100,000,000       6
5.000.000,00      5
40,000,000        4
                 ..
25,000,000        1
50,500,000        1
6,000,000,000     1
800.000.000,-     1
1,730,000,000     1
Name: 009_nominal_amount, Length: 96, dtype: int64

###	008_maturity_date	


In [0]:
df_['008_maturity_date'].value_counts()

4 October 2041        7
27 May 2036           5
24February 2020       3
9 August 2019         3
14 January 2021       2
                     ..
24 October 2039       1
7. August 2012        1
11. Juli 2011         1
29 January 2020       1
21. September 2012    1
Name: 008_maturity_date, Length: 129, dtype: int64

### 010_redemption_formula

In [0]:
df_['010_redemption_formula'].value_counts()

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          104
100%                                                                                                                                                                                                                                                      

### 017_first_payment_dat

In [0]:
df_['017_first_payment_date'].value_counts()

                     35
27 May 2019           4
10 November 2019      4
27 May 2018           3
4 October 2019        2
                     ..
 24 December 20l9     1
28. August 2016       1
10 August 2019        1
18 January 2020       1
8 September 2017      1
Name: 017_first_payment_date, Length: 100, dtype: int64

### 018_payment_dates

In [0]:
df_['018_payment_dates'].value_counts()

               13
4 October       8
27 May          5
29 January      3
10 February     3
               ..
08.07.          1
20. Juli        1
14 June         1
25.05.          1
22 May          1
Name: 018_payment_dates, Length: 111, dtype: int64

Checks whether GPU is available, switches to cuda if it is

In [0]:
is_using_gpu = spacy.prefer_gpu()
if is_using_gpu:
    print("Using GPU!")
    torch.set_default_tensor_type("torch.cuda.FloatTensor")
    print("GPU Usage")
    GPUtil.showUtilization()

Using GPU!
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |


## BERT

In [0]:
%%capture
!pip install ktrain

In [0]:
import ktrain
from ktrain import text

using Keras version: 2.2.4-tf


In [0]:
df_bert = df_[['pdf_text','003_product_subclass']]

In [0]:
# use pd.concat to join the new columns with your original dataframe
df_bert = pd.concat([df_bert,pd.get_dummies(df_bert['003_product_subclass'])],axis=1)

# now drop the original 'country' column (you don't need it anymore)
df_bert.drop(['003_product_subclass'],axis=1, inplace=True)
df_bert = df_bert[['pdf_text','Bond','Schuldschein']]#,'other Loan','Covered Bond']]

In [0]:
df_bert['is1'] = df_bert['Bond'] + df_bert['Schuldschein']
df_bert = df_bert[df_bert['is1']==1]
df_bert.shape

(94, 4)

In [0]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_df(df_bert, 
                                                                   'pdf_text', # name of column containing review text
                                                                   label_columns=['Bond','Schuldschein'],
                                                                   maxlen=500, 
                                                                   lang='ze',
                                                                   preprocess_mode='bert',
                                                                   val_pct=0.1)

downloading pretrained BERT model (multi_cased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: ze


preprocessing test...
language: ze


In [0]:

model = text.text_classifier('bert', (x_train, y_train) , preproc=preproc)
learner = ktrain.get_learner(model, 
                             train_data=(x_train, y_train), 
                             val_data=(x_test, y_test), 
                             batch_size=1)

Is Multi-Label? False
maxlen is 100


In [0]:
# using 1cycle learning rate schedule for 3 epochs
learner.fit_onecycle(2e-5, 2)